In [1]:
pip install transformers flask torch datasets nltk pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 57.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 48.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer, T5ForConditionalGeneration, T5Tokenizer, BertForSequenceClassification, BertTokenizer
from transformers import Trainer, TrainingArguments
from flask import Flask, request, jsonify
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from datasets import load_dataset

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
# Load the pre-trained models and tokenizers
chatbot_model = GPT2LMHeadModel.from_pretrained('gpt2')
chatbot_tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

summarization_model = T5ForConditionalGeneration.from_pretrained('t5-small')
summarization_tokenizer = T5Tokenizer.from_pretrained('t5-small')

sentiment_model = BertForSequenceClassification.from_pretrained('bert-base-uncased')
sentiment_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [4]:
# Example datasets (small subset for testing)
chatbot_data = [
    ("How can I reset my password?", "To reset your password, click on 'Forgot password' at the login screen."),
    ("Where can I find my order status?", "You can find your order status in your account under 'Order History'.")
]

summarization_data = [
    ("The global climate change issue is becoming more critical each day due to the rising levels of CO2.", "Summary of the article about climate change.")
]

sentiment_data = [
    ("I love this product, it's amazing!", 1),  # 1 for positive sentiment
    ("This is the worst purchase I've made.", 0)  # 0 for negative sentiment
]

# Convert to pandas DataFrame
chatbot_df = pd.DataFrame(chatbot_data, columns=["Question", "Answer"])
summarization_df = pd.DataFrame(summarization_data, columns=["Text", "Summary"])
sentiment_df = pd.DataFrame(sentiment_data, columns=["Text", "Sentiment"])

In [8]:
def fine_tune_chatbot(chatbot_data):
    # Add padding token to the tokenizer
    chatbot_tokenizer.pad_token = chatbot_tokenizer.eos_token # Use eos_token as pad_token

    # Tokenize inputs and labels with padding and truncation
    encodings = chatbot_tokenizer([q[0] for q in chatbot_data],
                                   text_pair=[q[1] for q in chatbot_data],  # Tokenize input-output pairs together
                                   return_tensors="pt",
                                   padding=True,
                                   truncation=True)

    # Ensure labels have the same shape as inputs and are on the same device
    encodings["labels"] = encodings["input_ids"].clone().detach()
    encodings.to(chatbot_model.device)  # Move all tensors to the model's device

    # Set labels to -100 where padding tokens are present in the input_ids
    encodings["labels"] = torch.where(encodings["attention_mask"] == 0, -100, encodings["labels"])

    # Forward pass
    outputs = chatbot_model(**encodings)  # Pass the encodings dictionary directly
    loss = outputs.loss
    return loss

fine_tune_chatbot(chatbot_data)

tensor(2.7128, grad_fn=<NllLossBackward0>)

In [9]:
def fine_tune_summarization(summarization_data):
    inputs = summarization_tokenizer(summarization_data["Text"].tolist(), return_tensors="pt", padding=True, truncation=True)
    labels = summarization_tokenizer(summarization_data["Summary"].tolist(), return_tensors="pt", padding=True, truncation=True).input_ids
    outputs = summarization_model(input_ids=inputs["input_ids"], labels=labels)
    loss = outputs.loss
    return loss

# Fine-tune the summarization model (simplified for demonstration)
fine_tune_summarization(summarization_df)


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


tensor(4.7516, grad_fn=<NllLossBackward0>)

In [10]:
def fine_tune_sentiment_analysis(sentiment_data):
    inputs = sentiment_tokenizer(sentiment_data["Text"].tolist(), return_tensors="pt", padding=True, truncation=True)
    labels = torch.tensor(sentiment_data["Sentiment"].tolist())
    outputs = sentiment_model(input_ids=inputs["input_ids"], labels=labels)
    loss = outputs.loss
    return loss

# Fine-tune the sentiment model (simplified for demonstration)
fine_tune_sentiment_analysis(sentiment_df)

tensor(0.7194, grad_fn=<NllLossBackward0>)

In [11]:
app = Flask(__name__)

# Chatbot endpoint
@app.route('/chatbot', methods=['POST'])
def chatbot():
    input_text = request.json['text']
    tokens = chatbot_tokenizer(input_text, return_tensors='pt')
    response_ids = chatbot_model.generate(tokens['input_ids'])
    response = chatbot_tokenizer.decode(response_ids[0], skip_special_tokens=True)
    return jsonify({'response': response})

# Summarization endpoint
@app.route('/summarize', methods=['POST'])
def summarize():
    input_text = request.json['text']
    inputs = summarization_tokenizer(input_text, return_tensors='pt', padding=True, truncation=True)
    summary_ids = summarization_model.generate(inputs["input_ids"])
    summary = summarization_tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return jsonify({'summary': summary})

# Sentiment analysis endpoint
@app.route('/sentiment', methods=['POST'])
def sentiment():
    input_text = request.json['text']
    inputs = sentiment_tokenizer(input_text, return_tensors='pt', padding=True, truncation=True)
    outputs = sentiment_model(**inputs)
    sentiment_score = torch.argmax(outputs.logits, dim=1).item()
    sentiment = 'Positive' if sentiment_score == 1 else 'Negative'
    return jsonify({'sentiment': sentiment})

if __name__ == '__main__':
    app.run(debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat


In [13]:
!pip install flask flask-ngrok


In [14]:
from flask import Flask, request, jsonify
from flask_ngrok import run_with_ngrok  # Import ngrok for tunneling

app = Flask(__name__)
run_with_ngrok(app)  # Enables ngrok

@app.route("/", methods=["GET"])
def home():
    return "Flask app is running!"

@app.route("/chatbot", methods=["POST"])
def chatbot():
    data = request.json
    user_input = data.get("text", "")
    response = {"reply": f"You said: {user_input}"}
    return jsonify(response)

if __name__ == "__main__":
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
Exception in thread Thread-10:
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/urllib3/connection.py", line 198, in _new_conn
    sock = connection.create_connection(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/urllib3/util/connection.py", line 85, in create_connection
    raise err
  File "/usr/local/lib/python3.11/dist-packages/urllib3/util/connection.py", line 73, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/urllib3/connectionpool.py", line 787, in urlopen
    response = self._make_request(
           